In [43]:
#https://towardsdatascience.com/efficient-frontier-optimize-portfolio-with-scipy-57456428323e
import pandas as pd
import statsmodels.api as sm
import numpy as np
from scipy.optimize import minimize,lsq_linear
import pulp
from function_helper import *
from statsmodels.formula.api import glm


df = pd.read_excel('Data/fundret.xlsx',skiprows=3,skipfooter=1)#.dropna()
df = df.dropna(axis=1,how='all').dropna(axis=0)
df['Month'] = pd.to_datetime(df['Month'],format="%Y%m")
#df.set_index('Month',inplace=True)
df = df.astype(float,errors='ignore')
russell = ['Return.1','R1000G', 'R1000V', 'R2000G','R2000V', 'T-Bill']

In [44]:
import numpy as np
from scipy.optimize import minimize

def con_solver(y,m):
    def loss(x):
        return np.var(np.dot(x, m) - y)
    cons = ({'type': 'eq',
             'fun' : lambda x: np.sum(x) - 1.0})
    x0 = np.zeros(m.shape[0])
    res = minimize(loss, x0, method='SLSQP', constraints=cons,
                   bounds=[(0, np.inf) for i in range(m.shape[0])], options={'disp': False,  'maxiter':100000})
    return (np.round(res.x,5),loss(res.x))#pd.DataFrame(np.round(res.x,5),index=['R1000G', 'R1000V', 'R2000G','R2000V', 'T-Bill'])
    #print(loss(res.x))
#print(np.dot(res.x, m))
#print(np.sum(np.square(np.dot(res.x, m) - y)))

In [45]:
from itertools import islice

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield np.array(result)
    for elem in it:
        result = result[1:] + (elem,)
        yield np.array(result)

In [46]:
M = df[['R1000G', 'R1000V', 'R2000G','R2000V', 'T-Bill']].values
Y = np.squeeze(df[['Month','Return.1']].values)
df_values = []
for y,m in zip(window(Y,n=36),window(M,n=36)):
     #print(y.shape,m.shape)
     solver = con_solver(y[:,1],np.transpose(m))
     df_values.append((y[-1,0],
                        solver[0],
                        solver[1],
                        1-(solver[1]/np.var(y[:,1]))))

In [47]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
answer = pd.DataFrame(df_values,columns=['date','weights','residual variance','Rsquared'])

In [52]:
pd.DataFrame((answer[answer['date'].astype(str)=='1999-12-01']['Rsquared'].values,
              answer[answer['date'].astype(str)=='2009-12-01']['Rsquared'].values,
              answer[answer['date'].astype(str)=='2019-12-01']['Rsquared'].values))

,0
0,0.930922
1,0.978915
2,0.935980


In [49]:
final_answer = pd.DataFrame(np.vstack([answer[answer['date'].astype(str)=='1999-12-01']['weights'].values[0],
                        answer[answer['date'].astype(str)=='2009-12-01']['weights'].values[0],
                        answer[answer['date'].astype(str)=='2019-12-01']['weights'].values[0]],),columns=['R1000G', 'R1000V', 'R2000G','R2000V', 'T-Bill'],index = [2000,2010,2020])